# Minitarea 2

Nombre: Joaquín Cubelli 

---------------------------
## Language Models (3 pts)
Estas preguntas son teóricas y deben ser resueltas con desarrollo, pero sin código. Por favor usen $\LaTeX$ para las fórmulas. El material del curso y los videos deberian ser suficientes para que puedan responder.


### Pregunta 1 (1 pt)
Asuma que tiene un modelo de lenguaje de trigramas (simple) entrenado sobre un corpus C. Muestre cómo el modelo le asigna una probabilidad a la oración `el perro le ladra al gato` usando los parámetros estimados a partir del corpus  $q(w_i | w_{i-2}, w_{i-1})$ .

**Respuesta:** 

Utilizando los tokens $*$ y *STOP*, se obtiene la siguiente probabilidad:

P(El,perro,le,ladra,al,gato) = $q(El|*,*)*q(perro|*,El)*q(le|el,perro)*q(ladra|perro,le)*q(al|le,ladra)*q(gato|ladra,al)*q(STOP|al,gato)$

### Pregunta 2 (1 pt)
Muestre cómo se calcularía  $q(w_{i} | w_{i-2}, w_{i-1})$  usando un modelo que interpola un modelo de lenguajes de trigramas, bigramas, y unigrama.


**Respuesta:**

$q(w_{i} | w_{i-2}, w_{i-1})$ = $λ_1 * q_{ML}(w_{i} | w_{i-2}, w_{i-1})$ + $λ_2 * q_{ML}(w_{i} | w_{i-1})$ + $λ_3 * q_{ML}(w_{i})$

Donde: 

$ q_{ML}(w_{i} | w_{i-2}, w_{i-1}) $ = $\frac{Count(w_{i-2},w_{i-1},w_{i})}{Count(w_{i-2},w_{i-1})}$

$ q_{ML}(w_{i} | w_{i-1}) $ = $\frac{Count(w_{i-1},w_{i})}{Count(w_{i-1})}$

$ q_{ML}(w_{i}) $ = $\frac{Count(w_{i})}{Count()}$

Y los λ se calculan buscando el mínimo Perplexity, dado que son hiperparámetros.

### Pregunta 3 (1 pt)
¿Qué ventajas tiene el modelo interpolado sobre el modelo de trigramas simple?

**Respuesta:**

Disminuye la sparsness, al hacer que tanto trigramas como bigramas que tendrían probabilidad cero bajo un modelo de trigramas simple porque no fueron encontradas en el corpus, tengan una probabilidad dada mayor a cero en base a sus bigramas y unigramas que lo conforman, permitiendo suavizar de cierta manera las probabilidades y generalizando mejor. 


-----------------------
## Naive Bayes (3 pts)
En esta parte de la minitarea deberan programar Naive Bayes Multinomial usando Laplace Smothing. Las referencias las pueden encontrar en el material del curso y los videos del profesor.

A continuacion se presentan un conjunto de documentos de training divididos en 2 categorias distintas. Ustedes deberan clasificar los documentos del test set usando Naive Bayes con Laplace Smothing.

Por favor, documenten su codigo. Escriban que hacen las funciones, que reciben, que entregan, etc. Si en algun lugar de su codigo hacen algo que creen que debe ser explicado, pongan comentarios, son bienvenidos. 


**Underflow prevention:** En vez de hacer muchas multiplicacions de `float`s, reemplacenlas por sumas de logaritmos para prevenir errores de precision. Revisen la diapo 69 de las slides. 

NOTA: Sobre las `namedtuple`s. Es el tipo de los documentos. Son objetos inmutable que tienen dos atributos: `words` donde estan las palabras del documento y `class_` donde se guarda la clase de ese documento. Estos objetos son inmutables, lo que quiere decir que si quieren modificar un documento y cambiarle la clase, tienen que crear otro documento. Otra cosa es que son tuplas como cualquier otra, es decir se pueden acceder usando indices como `doc[0]` o `doc[1]`. Son libres de implementar su solucion como quieran, si no les gusta este tipo de dato usen otro.



In [1]:
from collections import namedtuple
document = namedtuple(
    "document", ("words", "class_")  # avoid python's keyword collision
)

train_set = (
    document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0),
    document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1),
    document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0),
    document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1),
    document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1),
    document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=2),
    document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0),
    document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1)
)
print("Train documents:")
print(train_set)


test_set = (document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None),
            document(words=('w03', 'w06', 'w05', 'w02', 'w05', 'w02', 'w01'), class_=None),
            document(words=('w03', 'w03', 'w07', 'w01', 'w06', 'w09', 'w01', 'w08'), class_=None)
)
print("\nTest documents:")
print(test_set)

Train documents:
(document(words=('w03', 'w01', 'w02', 'w06', 'w02', 'w08', 'w07'), class_=0), document(words=('w05', 'w04', 'w00', 'w06', 'w09', 'w07', 'w06', 'w09', 'w05'), class_=1), document(words=('w07', 'w06', 'w00', 'w08', 'w01', 'w08', 'w08', 'w09', 'w02'), class_=0), document(words=('w08', 'w09', 'w02', 'w06', 'w05', 'w08', 'w07'), class_=1), document(words=('w09', 'w08', 'w05', 'w08', 'w05', 'w00', 'w08'), class_=1), document(words=('w05', 'w05', 'w06', 'w01', 'w06', 'w08', 'w02'), class_=2), document(words=('w04', 'w03', 'w07', 'w05', 'w04', 'w00', 'w02'), class_=0), document(words=('w01', 'w00', 'w01', 'w04', 'w09', 'w02', 'w04', 'w07'), class_=1))

Test documents:
(document(words=('w02', 'w09', 'w06', 'w01', 'w05', 'w04', 'w03', 'w03'), class_=None), document(words=('w03', 'w06', 'w05', 'w02', 'w05', 'w02', 'w01'), class_=None), document(words=('w03', 'w03', 'w07', 'w01', 'w06', 'w09', 'w01', 'w08'), class_=None))


In [2]:
import math # Se importa math para calcular logaritmos.

Primero, se crea el vocabulario en base al train set

In [3]:
def create_vocab(set): # Funcion que crea el vocabulario en base al train_set
    V = []
    for j in range(len(train_set)):
        for i in set[j][0]:
            V.append(i)
    return list(dict.fromkeys(V))

In [4]:
V = create_vocab(train_set)
V

['w03', 'w01', 'w02', 'w06', 'w08', 'w07', 'w05', 'w04', 'w00', 'w09']

Buscamos generalizar, por lo que buscamos en el train_set todos los nombres de las clases, es más, se modificó el train set y se creó una clase nueva para ver si generalizaba.

In [5]:
def classesNames(set):
    C = []
    for j in range(len(train_set)):
        i = set[j][1]
        C.append(i) # Se agregan todas los nombres de las clases a una lista.
    return (list(dict.fromkeys(C)))

In [6]:
C = classesNames(train_set)
C

[0, 1, 2]

In [7]:
def prob_c(set): 
    n = len(set)
    c = classesNames(set)
    doc_count = [0]*len(c) # Se genera una lista que va a ser usada para contar los documentos de cada clase.
    for j in range(n):
        cls = set[j][1] # Clase documento j.
        for i in range(len(c)):
            if cls == c[i]:
                doc_count[i]+=1
    for i in range(len(c)): # Una vez contados todos los documentos, se divide en el numero total, y se aplica el log.
        doc_count[i] = math.log(doc_count[i]/n)
    
    return doc_count

In [8]:
prob_c(train_set)

[-0.9808292530117262, -0.6931471805599453, -2.0794415416798357]

In [9]:
def prob_w_hyperword(set, vocab, classname): # Se cuentan todas las palabras de cada asociada a cada clase, y se
                                             # calcula su probabilidad
    C = []
    l = len(vocab)
    for i in set: 
        if i[1] is classname:
            for j in i[0]:
                C.append(j)
    Count = [1]*l
    log_probc = Count
    for i in range(0,l):
        for j in range(0,len(C)):
            if C[j]==vocab[i]:
                Count[i]+=1 
    Word_n = 0
    #print(Count)
    for i in Count:
        Word_n+=i
    #print(Word_n)
    for i in range(l):
        log_probc[i]=math.log(Count[i]/Word_n)
    return log_probc

In [16]:
prob_w_hyperword(train_set, V, 0)
#for i in prob_w_hyperword(train_set, V, 0):
#    print(math.exp(i))

[-2.3978952727983707,
 -2.3978952727983707,
 -1.8870696490323797,
 -2.3978952727983707,
 -1.8870696490323797,
 -2.1102132003465894,
 -2.803360380906535,
 -2.3978952727983707,
 -2.3978952727983707,
 -2.803360380906535]

In [19]:
def classify(test, train): 
    # Operaciones de train:
    vocab = create_vocab(train)
    p_c = prob_c(train)
    c = classesNames(train)
    nc = []
    for i in range(len(c)): # Se calculan las probabilidades de que cada palabra pertenezca a cada clase
        nc.append(prob_w_hyperword(train, vocab, i))
    # Operaciones en test:
    p = []
    for i in range(len(c)):
        p.append(prob_c(train))

    for i in range(len(test)):
        for j in test[i][0]:
            for m in range(len(c)):
                for k in range(len(vocab)):
                    if j == vocab[k]:
                        p[i][m] += nc[m][k]
    
    #for i in p:
        #print(i) #prueba, sirve para ver el máximo de las clases para cada oración.
    
    # Se entrega el resultado en base al valor más alto logaritmicamente en probabilidad.
    for i in range(len(test)):
        print("Oración "+str(i)+" corresponde a clase ",end="")
        for m in range(len(c)): 
            if p[i][m]==max(p[i]):
                print(c[m])
    

In [20]:
classify(test_set, train_set)

Oración 0 corresponde a clase 0
Oración 1 corresponde a clase 2
Oración 2 corresponde a clase 0


**Atención:** Se cambió el database para crear un código más generalizado, que pudiese procesar más data de prueba simultaneamente, y más clases.